# 关键字

## `with`

`with`关键字用于简化资源管理,类似`java`中的`try-with-resources`语句,确保在资源使用完成后能够自动执行清理代码,例如关闭文件、释放数据库连接等.

**语法**:

```python
with expression:ContextManager [as variable]:
    #with-block
```

**上下文管理器(Context Manager)**:

`上下文管理器`是一个实现了`__enter__`和`__exit__`方法的对象,这两个方法分别定义了进入和退出上下文时的行为

### 一个简单的例子

In [ ]:
with open('example.txt', 'r') as file:
    content = file.read()
    print(content)

### 自定义上下文管理器

In [1]:
class MyContextManager:
    def __enter__(self):
        print("Entering the context")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting the context")

# 使用自定义上下文管理器
with MyContextManager():
    print("Inside the context")

Entering the context
Inside the context
Exiting the context


# 模块化

## `__init__.py`


`__init__.py`文件用于以下目的:

1. **标识目录为Python的包** - 在`3.3`版本之前, Python只会将包含`__init__.py`文件的目录视为包, 从`3.3`版本开始, Python会将所有目录视为包, 但是为了向后兼容, 通常还是会在目录中包含`__init__.py`文件
2. **初始化包** - `__init__.py`文件可以包含包的初始化代码, 当包**第一次被导入**时, `__init__.py`文件会被自动执行,可以在这个文件中初始化包的模块、设置包级别的变量、导入子模块或子包

### 验证`__init__.py`只会在包第一次被导入时执行

下面这个例子,第一次运行会输出:

```text
__init__.py is running
```

第二次就不会输出了,说明`__init__.py`只会在包第一次被导入时执行

In [1]:
import  test_package

print(test_package.__version__)

__init__.py is running
1.0.0


In [1]:
import test_package

# 调用包级别的函数
test_package.package_function()

# 调用子模块的函数
test_package.function1()
test_package.function2()

# 使用 * 导入
from test_package import *
function1()
function2()
package_function()

__init__.py is running
This is a function defined at the package level.
Function 1 from module 1
Function 2 from module 2
Function 1 from module 1
Function 2 from module 2
This is a function defined at the package level.


## 热插拔

[pluginbase](https://github.com/mitsuhiko/pluginbase)这个库用于在`Python`中实现插件系统.

# 装饰器(Decorators)

装饰器是Python的一个强大特性,它可以让函数或类在不修改原有代码的情况下增加新的功能,装饰器本质上是一个函数,它接受一个函数作为参数,并返回一个新的函数.

## 一个简单的例子

下面这个例子定义了一个装饰器`my_decorator`,它接受一个函数作为参数,并返回一个新的函数,这个新的函数在调用原函数之前会输出`start`,在调用原函数之后会输出`end`

In [1]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Start")
        result = func(*args, **kwargs)
        print("End")
        return result
    return wrapper

@my_decorator
def add(a, b):
    print("Add")
    return a + b

print(add(1, 2))
# 上面的代码本质上是
# add = my_decorator(add)

Start
Add
End
3


## 多个装饰器

一个函数可以同时被多个装饰器装饰,装饰器的执行顺序是从下往上,即最后一个装饰器最先执行

## 装饰器参数

装饰器也可以接受参数,下面这个例子中的`repeat`装饰器,它接受一个参数`n`,表示重复调用原函数`n`次

In [16]:
def repeat(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")
# 上面的代码本质上是
# greet = repeat(3)(greet)

3
Hello, Alice!
Hello, Alice!
Hello, Alice!


In [7]:
def decorator1(func):
    def wrapper(*args, **kwargs):
        print("Decorator 1")
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    def wrapper(*args, **kwargs):
        print("Decorator 2")
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def my_function():
    print("Inside the function")

my_function()
# 上面的代码本质上是
# my_function = decorator1(decorator2(my_function))

Decorator 1
Decorator 2
Inside the function


## `wrapt`

[wrapt](https://wrapt.readthedocs.io/en/master/)是一个简化装饰器编写的库,它提供了`FunctionWrapper`和`ObjectProxy`两个类,可以用来编写更加复杂的装饰器


### 实现`my_decorator`的功能

下面这个例子使用`wrapt`库实现了上面提到的`my_decorator`的功能

In [3]:
import wrapt
@wrapt.decorator
def my_decorator(wrapped, instance, args, kwargs):
    print("Start")
    result = wrapped(*args, **kwargs)
    print("End")
    return result

@my_decorator
def add(a, b):
    print("Add")
    return a + b

print(add(1, 2))

Start
Add
End
3


### 多个装饰器

下面这个例子用`wrapt`重现了上面的`decorator1`和`decorator2`的功能

In [8]:
import wrapt

@wrapt.decorator
def decorator1(wrapped, instance, args, kwargs):
    print("Decorator 1")
    return wrapped(*args, **kwargs)

@wrapt.decorator
def decorator2(wrapped, instance, args, kwargs):
    print("Decorator 2")
    return wrapped(*args, **kwargs)

@decorator1
@decorator2
def my_function():
    print("Inside the function")

my_function()

Decorator 1
Decorator 2
Inside the function


### 装饰器参数

下面这个例子用`wrapt`实现了上面的`repeat`装饰器

In [18]:
import wrapt

def repeat(n):
    @wrapt.decorator
    def decorator(wrapped, instance, args, kwargs):
        print(f"Repeat {n} times")
        for _ in range(n):
            result = wrapped(*args, **kwargs)
        return result
    return decorator

@repeat(3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")

Repeat 3 times
Hello, Alice!
Hello, Alice!
Hello, Alice!


# 依赖管理

使用[pipreqs](https://pypi.org/project/pipreqs/)来自动生成`requirements.txt`文件

```bash

pip install pipreqs

pipreqs --force --ignore wslvenv,.venv .
    
```